In [3]:
import networkx as nx   
import axelrod as axl  

In [4]:
import numpy as np
import pandas as pd
import random
#import statsmodels.formula.api as sm

In [5]:
# The current winning strategies of a round robin tournament in axelrod library.
winning_strategies = [axl.Gambler(), axl.EvolvedLookerUp(), axl.DoubleCrosser(), axl.FoolMeOnce(),
           axl.Gradual(), axl.BackStabber(), axl.ForgetfulFoolMeOnce(), axl.MetaHunter(),
           axl.OmegaTFT(), axl.MetaMajorityLongMemory(), axl.MetaMajorityFiniteMemory(),
           axl.Grudger(), axl.MetaMajorityMemoryOne(), axl.MetaWinner(), axl.NiceAverageCopier(),
           axl.Shubik(), axl.ZDGTFT2(), axl.SoftGrudger(), axl.SoftJoss(), axl.LimitedRetaliate2()]

In [6]:
# ordinary strategies of axelrod
ordinary_players = [s() for s in axl.ordinary_strategies]

In [7]:
# random strategies of the ordinary strategies
random_players = random.sample(ordinary_players, 10)

In [8]:
def Neighbors(G):
    """
    Returns a list with the neighbors of a player.
    
    By neighbors we mean the players a player
    interacts with.
    """
    Neighbors = []
    for i in range(len(players)) : Neighbors.append(G.neighbors(i))
    return Neighbors

In [9]:
def Neighborhood_size (Neighborhood) :
    """
    Returns the size of the neighborhood of a
    player.
    """
    Neighborhood_size = []
    for i in range(len(Neighborhood)) : Neighborhood_size.append(len(Neighborhood[i]))
    return Neighborhood_size

In [10]:
def Neighbors_Scores(G, results):
    """
    Returns a list with the scores of the 
    neighbors.
    """
    return [[results.scores[j] for j in G.neighbors(i)]
            for i, pl in enumerate(players)]

In [11]:
def Average_Neighborhood_Score(G, results) :
    """
    Returns the average score of the
    sum of the neighbors scores.
    
    Thus, the average score of the neighborhood.    
    """
    av_score = []
    temp = [[j for m in G.neighbors(k) for j in results.scores[m]]
              for k in G.nodes()]
    for i in G.nodes() :
        av_score.append(np.mean(temp[i]))
    return av_score

In [19]:
def Cliques(G) :
    nx.find_cliques(G)
    
    cliques = []
    for i in G.nodes() : 
        cliques.append(nx.cliques_containing_node(G, i))
    return cliques
    

In [20]:
def touranment(G, p, seed):
    """
    Run a spatial tournament with a topology
    of any given graph. Return the results.
    """
    # set seed the tournament. 
    random.seed(seed)
    np.random.seed(seed)
    edges = G.edges()
    
    # create tournament
    tournament = axl.SpatialTournament(players, edges=G.edges(), turns = 200, repetitions=5)
    
    # play the tournament. Return the results.
    return tournament.play(processes=0)
                           #filename="/home/nikoleta/src/Dissertation-Notebooks/Data/Watts Strogatz/Watts_Strogatz-8-{}-{}.csv".format(p, seed))

In [21]:
def tournament_results(G, seed,p):
    """
    Creates a data frame with parameters of the tournament.
    
    Parameters
    ----------
    players_list : list
            A list with the players participating in the tournament
    seed : integer
            A seed for the tournament and graph
    player_index : integer
            A players index number
    player_name : character
            The name of the player           
    degree : integer
            THe degree of the graph
    neighbors : list
            A list with the player's neighbors index number
    neighborhood_size: integer
            The size of the neighborhood, should be eqaul with the degree
   
    """
    # generate the tournament and the results
    results = touranment(G, p, seed)
    
    # parameters 
    neighborhood = Neighbors(G)
    nsize = Neighborhood_size(neighborhood)
    degree = list(G.degree(G.nodes()).values())
    nscores = Neighbors_Scores(G, results)
    av_nscores = Average_Neighborhood_Score(G, results)
    cliques = Cliques(G)
    
    
    # create data frame
    data = pd.DataFrame({'players_list' : [players for _ in results.players],
                         'seed' : seed, 'parameter': p ,'player_index' : G.nodes(),
                         'player_name' : results.players, 'degree' : degree ,
                         'neighbors' : neighborhood, 'neighborhood_size' : nsize,
                         'ranking' : results.ranking, 'scores' : results.scores,
                         'normalised_scores' : results.normalised_scores,
                         'average_score' : [np.median(scores) for scores in results.normalised_scores],
                         'neighbors_scores' : nscores, 'average_neighboorhood_score' : av_nscores,
                         'R' : [list(results.game.RPST())[0] for _ in results.players],
                         'P' : [list(results.game.RPST())[1] for _ in results.players],
                         'S' : [list(results.game.RPST())[2] for _ in results.players],
                         'T' : [list(results.game.RPST())[3] for _ in results.players],
                         'connectivity' : nx.node_connectivity(G), 
                         'clustering' : nx.average_clustering(G),
                         'cliques' : cliques
                        })
   
    return data

In [68]:
nx.find_cliques(G)

In [80]:
for i in G.nodes() : k = nx.cliques_containing_node(G, i)

In [81]:
k

[[0, 128, 129, 130, 131],
 [0, 1, 129, 130, 131],
 [0, 1, 2, 130, 131],
 [0, 1, 2, 3, 131],
 [127, 128, 129, 130, 131]]

In [71]:
nx.enumerate_all_cliques(G)

<generator object enumerate_all_cliques at 0x7fd78e6a9048>

In [22]:
# define the players
players = random_players

In [23]:
cols = ['players_list', 'seed', 'parameter', 'player_name', 'player_index', 
        'degree', 'neighbors', 'neighborhood_size', 'ranking', 'scores',
        'normalised_scores',  'average_score', 'neighbors_scores',
        'average_neighboorhood_score', 'R', 'P', 'S', 'T', 
        'clustering', 'connectivity', 'cliques']

In [17]:
# for the binomial graphs
while nodes != list(range(len(players))): 
    G = nx.binomial_graph(len(players), p=.1)
    nodes = list(set([node for edge in G.edges() for node in edge]))

NameError: name 'nodes' is not defined

In [24]:
results = pd.DataFrame()
for seed in range(0, 1):
    
    np.random.seed(seed)
    p = seed/100
    
    # define the graph 
    G = nx.watts_strogatz_graph (len(players), 8, p)
    
    results = results.append([tournament_results(G, seed,p)])

results = results[cols]
#results.to_csv("/home/nikoleta/src/Dissertation-Notebooks/Data/Watts Strogatz/Watts_Strogatz_Eight_Tournament.csv", index=False) 


In [25]:
results

,players_list,seed,parameter,player_name,player_index,degree,neighbors,neighborhood_size,ranking,scores,...,average_score,neighbors_scores,average_neighboorhood_score,R,P,S,T,clustering,connectivity,cliques
0,"[Limited Retaliate (0.05/20), Meta Majority Me...",0,0.0,Limited Retaliate (0.05/20),0,8,"[1, 2, 3, 4, 6, 7, 8, 9]",8,2,"[3514, 3538, 3515, 3529, 3517]",...,2.198125,"[[3484, 3841, 3761, 3756, 3964], [4059, 4231, ...",3500.075,3,1,0,5,0.857143,8,"[[0, 1, 2, 8, 9], [0, 1, 2, 8, 4], [0, 1, 2, 3..."
1,"[Limited Retaliate (0.05/20), Meta Majority Me...",0,0.0,Meta Majority Memory One,1,8,"[0, 2, 3, 4, 5, 7, 8, 9]",8,9,"[3484, 3841, 3761, 3756, 3964]",...,2.350625,"[[3514, 3538, 3515, 3529, 3517], [4059, 4231, ...",3381.650,3,1,0,5,0.857143,8,"[[0, 1, 2, 8, 9], [0, 1, 2, 8, 4], [0, 1, 2, 3..."
2,"[Limited Retaliate (0.05/20), Meta Majority Me...",0,0.0,Prober,2,8,"[0, 1, 3, 4, 5, 6, 8, 9]",8,6,"[4059, 4231, 4321, 4327, 4031]",...,2.644375,"[[3514, 3538, 3515, 3529, 3517], [3484, 3841, ...",3344.150,3,1,0,5,0.857143,8,"[[0, 1, 2, 8, 9], [0, 1, 2, 8, 4], [0, 1, 2, 3..."
3,"[Limited Retaliate (0.05/20), Meta Majority Me...",0,0.0,Predator,3,8,"[0, 1, 2, 4, 5, 6, 7, 9]",8,1,"[2491, 2489, 2510, 2496, 2491]",...,1.556875,"[[3514, 3538, 3515, 3529, 3517], [3484, 3841, ...",3632.200,3,1,0,5,0.857143,8,"[[0, 1, 2, 3, 9], [0, 1, 2, 3, 4], [0, 1, 7, 3..."
4,"[Limited Retaliate (0.05/20), Meta Majority Me...",0,0.0,Alternator Hunter,4,8,"[0, 1, 2, 3, 5, 6, 7, 8]",8,7,"[2802, 2793, 2772, 2781, 2796]",...,1.745625,"[[3514, 3538, 3515, 3529, 3517], [3484, 3841, ...",3487.025,3,1,0,5,0.857143,8,"[[0, 1, 2, 8, 4], [0, 1, 2, 3, 4], [0, 1, 7, 8..."
5,"[Limited Retaliate (0.05/20), Meta Majority Me...",0,0.0,ZD-Extort-4,5,8,"[1, 2, 3, 4, 6, 7, 8, 9]",8,0,"[3162, 3171, 3175, 3207, 3120]",...,1.981875,"[[3484, 3841, 3761, 3756, 3964], [4059, 4231, ...",3500.075,3,1,0,5,0.857143,8,"[[5, 1, 2, 8, 9], [5, 1, 2, 8, 4], [5, 1, 2, 3..."
6,"[Limited Retaliate (0.05/20), Meta Majority Me...",0,0.0,Calculator,6,8,"[0, 2, 3, 4, 5, 7, 8, 9]",8,8,"[4176, 3794, 3766, 3794, 3849]",...,2.371250,"[[3514, 3538, 3515, 3529, 3517], [4059, 4231, ...",3381.650,3,1,0,5,0.857143,8,"[[0, 6, 2, 8, 9], [0, 6, 2, 8, 4], [0, 6, 2, 3..."
7,"[Limited Retaliate (0.05/20), Meta Majority Me...",0,0.0,Fortress3,7,8,"[0, 1, 3, 4, 5, 6, 8, 9]",8,5,"[3672, 3856, 3651, 3650, 3887]",...,2.295000,"[[3514, 3538, 3515, 3529, 3517], [3484, 3841, ...",3344.150,3,1,0,5,0.857143,8,"[[0, 1, 7, 8, 9], [0, 1, 7, 8, 4], [0, 1, 7, 3..."
8,"[Limited Retaliate (0.05/20), Meta Majority Me...",0,0.0,Cycle Hunter,8,8,"[0, 1, 2, 4, 5, 6, 7, 9]",8,4,"[2799, 3325, 3360, 3352, 2850]",...,2.078125,"[[3514, 3538, 3515, 3529, 3517], [3484, 3841, ...",3632.200,3,1,0,5,0.857143,8,"[[0, 1, 2, 8, 9], [0, 1, 2, 8, 4], [0, 1, 7, 8..."
9,"[Limited Retaliate (0.05/20), Meta Majority Me...",0,0.0,Gradual,9,8,"[0, 1, 2, 3, 5, 6, 7, 8]",8,3,"[3980, 4004, 3992, 4058, 3992]",...,2.495000,"[[3514, 3538, 3515, 3529, 3517], [3484, 3841, ...",3487.025,3,1,0,5,0.857143,8,"[[0, 1, 2, 8, 9], [0, 1, 2, 3, 9], [0, 1, 7, 8..."
